In [1]:
from cloudvolume import CloudVolume
from cloudvolume.exceptions import SkeletonDecodeError
from itertools import islice
from skimage.transform import downscale_local_mean
import napari
from skimage import io
import random
import h5py
from skimage import measure
from brainlit.preprocessing import removeSmallCCs
import numpy as np 
import matplotlib.pyplot as plt 
import subprocess
import tables
from napari_animation import AnimationWidget
from tqdm import tqdm
import pickle
import networkx as nx
import seaborn as sns
import pandas as pd
import brainrender
from brainlit.utils.session import NeuroglancerSession
from skimage.filters import threshold_otsu, threshold_local
from brainlit.preprocessing import split_frags, rename_states_consecutively, label_points
from brainlit.algorithms.connect_fragments import most_probable_neuron_path
from brainlit.algorithms.connect_fragments import trace_evaluation
import similaritymeasures
from brainlit.viz import Bresenham3D
from cloudvolume import Skeleton
from sklearn.metrics import pairwise_distances_argmin_min
import time
%gui qt5

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


embedWindow(verbose=True): could not load ipyvtk_simple try:
> pip install -U git+https://github.com/Kitware/ipyvtk-simple.git


In [5]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"

ngl_sess = NeuroglancerSession(mip = 0, url = dir, url_segments=dir_segments)

res = [0.3,0.3,1]
threshold = 0.9

block_size = 101

img, bbox, vox = ngl_sess.pull_vertex_list(2, [0], [1,1,1])
bbox

Bbox([12875, 12476, 4476],[12878, 12479, 4479], dtype=int32)

In [7]:
dir = "s3://open-neurodata/brainlit/brain1"

vol = CloudVolume(dir, parallel=1, mip=0, fill_missing=True)
vol.shape

(33792, 25600, 13312, 1)

In [ ]:
#view
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img)
viewer.add_points(vox, size=7)
napari.run()

In [3]:
io.imsave("/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/500.tif", img)

Downloading: 100%|██████████| 1980/1980 [07:38<00:00,  8.56it/s]

In [ ]:
pred_path ="/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/250_Probabilities.h5"
f = h5py.File(pred_path, 'r')
pred = f.get('exported_data')
pred = pred[:,:,:,1]
im_processed = pred

mask = im_processed >= threshold
labels = measure.label(mask)

soma_coords = vox[0:1,:]

In [ ]:
new_labels = split_frags(soma_coords, labels, im_processed, threshold, res)
_, soma_lbls = label_points(new_labels, soma_coords, res)
soma_lbl = soma_lbls[0]

In [ ]:
viterbi = most_probable_neuron_path(image=img.astype(float), labels=new_labels, soma_lbls=[soma_lbl], resolution=res, coef_dist=10, coef_curv=1000)
viterbi.frags_to_lines()
viterbi.reset_dists(type="all")
viterbi.compute_all_costs_dist(point_point_func=viterbi.point_point_dist, point_blob_func=viterbi.point_blob_dist)
viterbi.compute_all_costs_int()
viterbi.create_nx_graph()

In [ ]:
with open('/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/viterbi_250.pickle', 'wb') as handle:
    pickle.dump(viterbi, handle)

In [ ]:
from brainlit.utils.Neuron_trace import NeuronTrace

nt = NeuronTrace(path="/Users/thomasathey/Documents/mimlab/mouselight/input/mim_slides/SNT_Data.swc")
df = nt.get_df()
df[['x','y','z']].to_numpy()

In [3]:
dir = "s3://open-neurodata/brainlit/bil1/"
dir = "precomputed://https://dlab-colm.neurodata.io/2021_07_01_Sert_Cre_B/Ch_647"

vol = CloudVolume(dir, mip=0, parallel=1, fill_missing=True)

print(vol.shape)
vol[1000:1010,1000:1010,1000:1010]


(5602, 8800, 4200, 1)


Downloading: 100%|██████████| 10/10 [00:00<00:00, 31.02it/s]


VolumeCutout([[[[ 85],
                [ 96],
                [ 95],
                [ 97],
                [103],
                [124],
                [108],
                [ 97],
                [ 87],
                [101]],

               [[114],
                [108],
                [106],
                [104],
                [ 92],
                [104],
                [ 84],
                [104],
                [ 84],
                [ 96]],

               [[ 95],
                [106],
                [ 92],
                [121],
                [104],
                [101],
                [124],
                [111],
                [ 92],
                [ 89]],

               [[100],
                [120],
                [124],
                [102],
                [ 91],
                [ 97],
                [ 92],
                [102],
                [ 97],
                [ 85]],

               [[101],
                [ 97],
                [101],
                [ 94],
                [114],
                [107],
                [ 97],
                [112],
                [ 84],
                [ 95]],

               [[ 86],
                [114],
                [105],
                [ 93],
                [ 82],
                [101],
                [109],
                [ 98],
                [ 97],
                [116]],

               [[105],
                [112],
                [110],
                [104],
                [101],
                [ 99],
                [ 75],
                [117],
                [111],
                [113]],

               [[111],
                [105],
                [122],
                [105],
                [ 94],
                [101],
                [113],
                [ 97],
                [ 96],
                [ 89]],

               [[ 92],
                [121],
                [ 88],
                [117],
                [ 88],
                [103],
                [101],
                [ 99],
                [102],
                [104]],

               [[114],
                [107],
                [ 88],
                [ 99],
                [ 97],
                [ 90],
                [124],
                [104],
                [102],
                [109]]],


              [[[120],
                [ 87],
                [ 97],
                [101],
                [ 92],
                [ 88],
                [ 95],
                [ 80],
                [110],
                [ 79]],

               [[103],
                [106],
                [107],
                [ 94],
                [ 97],
                [ 89],
                [ 97],
                [ 95],
                [116],
                [ 90]],

               [[118],
                [107],
                [100],
                [117],
                [117],
                [113],
                [ 85],
                [110],
                [104],
                [101]],

               [[101],
                [100],
                [120],
                [ 92],
                [101],
                [110],
                [ 96],
                [107],
                [ 88],
                [ 92]],

               [[113],
                [102],
                [ 95],
                [113],
                [ 94],
                [ 86],
                [116],
                [101],
                [ 99],
                [ 91]],

               [[114],
                [103],
                [113],
                [123],
                [108],
                [113],
                [ 93],
                [101],
                [104],
                [ 96]],

               [[ 98],
                [110],
                [111],
                [106],
                [110],
                [102],
                [ 97],
                [108],
                [111],
                [116]],

               [[103],
                [114],
        